<a href="https://colab.research.google.com/github/sheensta/retail_products_ensemble_deep_learning/blob/main/NLP_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/df_NLP.csv')
df['description_clean'] = df['description_clean'].astype(str)
corpus = df['description_clean']

#https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/

Word Similarity with word2vec

In [ ]:
from gensim.models import word2vec
import nltk

In [ ]:
feature_size = 100    # Word vector dimensionality  
window_context = 30          # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in corpus]

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, 
                          window=window_context, min_count=min_word_count,
                          sample=sample, iter=50)

In [ ]:
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

w2v_feature_array = averaged_word_vectorizer(corpus=tokenized_corpus, model=w2v_model,
                                             num_features=feature_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [ ]:
pd.DataFrame(w2v_feature_array).shape

(51410, 100)

In [ ]:
similar_words = {search_term: [item[0] for item in w2v_model.wv.most_similar([search_term], topn=5)]
                  for search_term in ['pet','electronics','beauty','industrial','baby','arts','outdoors']}
similar_words

{'arts': ['imageaxs', 'alvin', 'crafts', 'drafting', 'widespread'],
 'baby': ['babies', 'child', 'infants', 'toddlers', 'mommy'],
 'beauty': ['indulges', 'singapour', 'traditions', 'passion', 'kate'],
 'electronics': ['automobile', 'lgic', 'pdas', 'lgsp', 'patibility'],
 'industrial': ['commercial',
  'dependable',
  'professional',
  'leading',
  'contractor'],
 'outdoors': ['indoors', 'outdoor', 'rv', 'porch', 'tablecloth'],
 'pet': ['pets', 'dog', 'stateslocated', 'cage', 'ferret']}

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
#le = LabelEncoder()
#le.fit(df['categories'])
#y = list(le.transform(df['categories']))


#num_classes = 21
y = list(df['categories'])
X = w2v_feature_array
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
w2v_feature_array = pd.DataFrame(X)
a = pd.concat([w2v_feature_array, y], axis = 1)
pd.DataFrame(a).to_csv('/content/drive/MyDrive/Colab Notebooks/w2v_feature_array_evaluation.csv', index= False)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
param_grid = [{}]
lg = GridSearchCV(LogisticRegression(), 
                           param_grid,
                           cv=KFold(n_splits=10, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=1)
y_preds_lg = lg.fit(X_train, y_train).predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 1 candidates, totalling 10 fits


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report
report_lg = classification_report( y_test, y_preds_lg)
print(report_lg)

NameError: ignored

In [ ]:
lg.score(X_test, y_test)

0.6761067455068855

In [ ]:
param_grid = [{}]
rf = GridSearchCV(RandomForestClassifier(), 
                           param_grid,
                           cv=KFold(n_splits=10, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=1)
y_preds_rf = rf.fit(X_train, y_train).predict_proba(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  6.8min finished


In [ ]:
len(X_test)

15423

In [ ]:
from sklearn.metrics import classification_report
report_rf = classification_report( y_test, y_preds_rf)
print(report_rf)

ValueError: ignored

In [ ]:
pd.DataFrame(y_preds_rf).to_csv('/content/drive/MyDrive/Colab Notebooks/y_preds_rf.csv', index = False)

In [ ]:
import joblib
joblib.dump(rf.best_estimator_, 'NLP_rf.pkl')

['NLP_rf.pkl']

In [ ]:
rf.best_estimator_.predict(X_test).shape

(12853, 21)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
le = LabelEncoder()
le.fit(df['categories'])
y = list(le.transform(df['categories']))


num_classes = 21

X = w2v_feature_array
y = list(le.transform(df['categories']))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
X_train = np.array(X_train)

In [ ]:
#xgboost hyperparameter tuning
import xgboost as xgb
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score
X = pd.DataFrame(w2v_feature_array)
xgb = XGBClassifier(objective = 'multiclass:softmax')

param_grid = [{}]
clf_xgb = GridSearchCV(xgb, 
                           param_grid,
                           cv=KFold(n_splits=10, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=1)
y_preds_xbg = clf_xgb.fit(X_train, y_train).predict_proba(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 61.2min finished


In [ ]:
pd.DataFrame(y_preds_xbg).to_csv('/content/drive/MyDrive/Colab Notebooks/y_preds_xgb.csv')

In [ ]:
joblib.dump(clf_xgb.best_estimator_, '/content/drive/MyDrive/models/NLP_XGB.pkl')

['/content/drive/MyDrive/models/NLP_XGB.pkl']

In [ ]:
report_xgb = classification_report( y_test, y_preds_xbg)
print(report_rf)

ValueError: ignored

In [ ]:
import joblib
filename = 'NLP_xgb.sav'
joblib.dump(clf_xgb, filename)



TypeError: ignored

Keras Word Preprocessing

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

X = list(df['description_clean'])
le = LabelEncoder()
le.fit(df['categories'])
y = list(le.transform(df['categories']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

import keras
num_classes = 21
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(corpus)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1 

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 250
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
#TRAINING A SIMPLE DEEP LEARNING MODEL (accuracy similar to Random Forest)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(Flatten())
model.add(Dense(21, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          7671800   
_________________________________________________________________
flatten (Flatten)            (None, 25000)             0         
_________________________________________________________________
dense (Dense)                (None, 21)                525021    
Total params: 8,196,821
Trainable params: 8,196,821
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_data=(X_test, y_test))

Epoch 1/6
282/282 [==============================] - 20s 63ms/step - loss: 2.7297 - accuracy: 0.2066 - val_loss: 1.3747 - val_accuracy: 0.6208
Epoch 2/6
282/282 [==============================] - 17s 62ms/step - loss: 1.1011 - accuracy: 0.7126 - val_loss: 1.0382 - val_accuracy: 0.6984
Epoch 3/6
282/282 [==============================] - 17s 62ms/step - loss: 0.7047 - accuracy: 0.8162 - val_loss: 0.9651 - val_accuracy: 0.7174
Epoch 4/6
282/282 [==============================] - 18s 62ms/step - loss: 0.5127 - accuracy: 0.8711 - val_loss: 0.9496 - val_accuracy: 0.7221
Epoch 5/6
282/282 [==============================] - 17s 62ms/step - loss: 0.3848 - accuracy: 0.9091 - val_loss: 0.9662 - val_accuracy: 0.7215
Epoch 6/6
282/282 [==============================] - 17s 62ms/step - loss: 0.3005 - accuracy: 0.9332 - val_loss: 0.9915 - val_accuracy: 0.7211


In [ ]:
model.save('/content/drive/MyDrive/models/NLP_custom_trainedsimple DL.h5')

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
#Embedding with pre-trained glove model

model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=True)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 100)          7671800   
_________________________________________________________________
flatten_3 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 21)                525021    
Total params: 8,196,821
Trainable params: 8,196,821
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_data=(X_test, y_test))

Epoch 1/6
282/282 [==============================] - 14s 49ms/step - loss: 2.2005 - accuracy: 0.3708 - val_loss: 1.4932 - val_accuracy: 0.5622
Epoch 2/6
282/282 [==============================] - 13s 48ms/step - loss: 1.0514 - accuracy: 0.7092 - val_loss: 1.3212 - val_accuracy: 0.6069
Epoch 3/6
282/282 [==============================] - 14s 48ms/step - loss: 0.7576 - accuracy: 0.8019 - val_loss: 1.2629 - val_accuracy: 0.6278
Epoch 4/6
282/282 [==============================] - 14s 48ms/step - loss: 0.5929 - accuracy: 0.8513 - val_loss: 1.2441 - val_accuracy: 0.6375
Epoch 5/6
282/282 [==============================] - 13s 46ms/step - loss: 0.4799 - accuracy: 0.8826 - val_loss: 1.2534 - val_accuracy: 0.6470
Epoch 6/6
282/282 [==============================] - 13s 45ms/step - loss: 0.4016 - accuracy: 0.9063 - val_loss: 1.2832 - val_accuracy: 0.6461


In [ ]:
model.save('/content/drive/MyDrive/models/NLP_GloVeEmbedding.h5')

In [ ]:
#using a CNN custom
from keras.layers import Conv1D 
from keras.layers import GlobalMaxPooling1D

model = Sequential()

model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_data=(X_test, y_test))

Epoch 1/6
282/282 [==============================] - 33s 64ms/step - loss: 2.7233 - accuracy: 0.2284 - val_loss: 1.3316 - val_accuracy: 0.6099
Epoch 2/6
282/282 [==============================] - 18s 63ms/step - loss: 1.1230 - accuracy: 0.6731 - val_loss: 1.0235 - val_accuracy: 0.6908
Epoch 3/6
282/282 [==============================] - 18s 63ms/step - loss: 0.7492 - accuracy: 0.7855 - val_loss: 0.9650 - val_accuracy: 0.7098
Epoch 4/6
282/282 [==============================] - 18s 63ms/step - loss: 0.5376 - accuracy: 0.8504 - val_loss: 0.9709 - val_accuracy: 0.7172
Epoch 5/6
282/282 [==============================] - 18s 63ms/step - loss: 0.3876 - accuracy: 0.8984 - val_loss: 0.9997 - val_accuracy: 0.7178
Epoch 6/6
282/282 [==============================] - 18s 63ms/step - loss: 0.2868 - accuracy: 0.9335 - val_loss: 1.0487 - val_accuracy: 0.7185


In [ ]:
model.save('/content/drive/MyDrive/models/NLP_custom_CNN.h5')

In [ ]:
#using a CNN with GloVe embedding
from keras.layers import Conv1D 
from keras.layers import GlobalMaxPooling1D

model = Sequential()

embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
282/282 [==============================] - 4s 11ms/step - loss: 2.2893 - accuracy: 0.3340 - val_loss: 1.5116 - val_accuracy: 0.5457
Epoch 2/20
282/282 [==============================] - 3s 10ms/step - loss: 1.3680 - accuracy: 0.5970 - val_loss: 1.3594 - val_accuracy: 0.5916
Epoch 3/20
282/282 [==============================] - 3s 10ms/step - loss: 1.1507 - accuracy: 0.6614 - val_loss: 1.2818 - val_accuracy: 0.6134
Epoch 4/20
282/282 [==============================] - 3s 10ms/step - loss: 1.0083 - accuracy: 0.7038 - val_loss: 1.2395 - val_accuracy: 0.6263
Epoch 5/20
282/282 [==============================] - 3s 10ms/step - loss: 0.9150 - accuracy: 0.7355 - val_loss: 1.2314 - val_accuracy: 0.6304
Epoch 6/20
282/282 [==============================] - 3s 10ms/step - loss: 0.8307 - accuracy: 0.7622 - val_loss: 1.2311 - val_accuracy: 0.6334
Epoch 7/20
282/282 [==============================] - 3s 10ms/step - loss: 0.7472 - accuracy: 0.7874 - val_loss: 1.2226 - val_accuracy: 0.6384

In [ ]:
model.save('/content/drive/MyDrive/models/NLP_GloVe_CNN.h5')

In [ ]:
#Using LSTM model with custom embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
1125/1125 [==============================] - 74s 64ms/step - loss: 3.0485 - accuracy: 0.0489 - val_loss: 3.0388 - val_accuracy: 0.0530
Epoch 2/10
1125/1125 [==============================] - 71s 63ms/step - loss: 3.0335 - accuracy: 0.0528 - val_loss: 3.0328 - val_accuracy: 0.0525
Epoch 3/10
1125/1125 [==============================] - 70s 62ms/step - loss: 3.0237 - accuracy: 0.0558 - val_loss: 3.0290 - val_accuracy: 0.0550
Epoch 4/10
 267/1125 [======>.......................] - ETA: 51s - loss: 3.0189 - accuracy: 0.0610

KeyboardInterrupt: ignored

In [ ]:
#Using LSTM model with GloVe embedding

model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(128))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))

Create model

In [ ]:
train_embedding_weights = X_train

In [ ]:
from keras.layers import concatenate
from keras.layers import Input
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.models import Model

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, trainable=False, extra_conv=False):
    
    embedding_layer = Embedding(vocab_size, 100, input_length=maxlen)
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate([convs[0],convs[1],convs[2]],axis=1)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(21,activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    model.summary()
    return model


In [ ]:
model = ConvNet(train_embedding_weights, maxlen, 35987, 250, False)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 250)]             0         
_________________________________________________________________
embedding_22 (Embedding)     (None, 250, 100)          7671800   
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 248, 128)          38528     
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 82, 128)           0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 82, 128)           0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 10496)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               1343

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64,validation_data=(X_test,y_test))

Epoch 1/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0402 - accuracy: 0.0614 - val_loss: 3.0394 - val_accuracy: 0.0615
Epoch 2/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0399 - accuracy: 0.0625 - val_loss: 3.0390 - val_accuracy: 0.0608
Epoch 3/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0397 - accuracy: 0.0607 - val_loss: 3.0385 - val_accuracy: 0.0612
Epoch 4/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0389 - accuracy: 0.0630 - val_loss: 3.0381 - val_accuracy: 0.0619
Epoch 5/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0387 - accuracy: 0.0611 - val_loss: 3.0376 - val_accuracy: 0.0616
Epoch 6/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0383 - accuracy: 0.0636 - val_loss: 3.0371 - val_accuracy: 0.0615
Epoch 7/50
563/563 [==============================] - 6s 10ms/step - loss: 3.0380 - accuracy: 0.0613 - val_loss: 3.0366 - val_accuracy: 0.0625

KeyboardInterrupt: ignored

In [ ]:
#Trying another model - also 5%, very slow to train

from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(GRU(units = 32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(21, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
#another CNN model, custom
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=6, batch_size=128,validation_data=(X_test,y_test))

Epoch 1/6
282/282 [==============================] - 18s 60ms/step - loss: 2.8442 - accuracy: 0.1129 - val_loss: 1.9639 - val_accuracy: 0.4180
Epoch 2/6
282/282 [==============================] - 17s 59ms/step - loss: 1.6742 - accuracy: 0.5175 - val_loss: 1.4534 - val_accuracy: 0.5939
Epoch 3/6
282/282 [==============================] - 16s 58ms/step - loss: 1.1528 - accuracy: 0.6674 - val_loss: 1.3935 - val_accuracy: 0.6152
Epoch 4/6
282/282 [==============================] - 16s 58ms/step - loss: 0.9749 - accuracy: 0.7141 - val_loss: 1.4040 - val_accuracy: 0.6196
Epoch 5/6
282/282 [==============================] - 16s 58ms/step - loss: 0.8483 - accuracy: 0.7524 - val_loss: 1.4528 - val_accuracy: 0.6236
Epoch 6/6
282/282 [==============================] - 16s 58ms/step - loss: 0.7349 - accuracy: 0.7835 - val_loss: 1.4976 - val_accuracy: 0.6221


In [ ]:
#another CNN model, GloVe
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=True))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=256,validation_data=(X_test,y_test))

Epoch 1/10
141/141 [==============================] - 10s 67ms/step - loss: 2.8849 - accuracy: 0.1165 - val_loss: 2.1310 - val_accuracy: 0.3697
Epoch 2/10
141/141 [==============================] - 9s 65ms/step - loss: 1.8897 - accuracy: 0.4295 - val_loss: 1.6173 - val_accuracy: 0.5116
Epoch 3/10
141/141 [==============================] - 9s 65ms/step - loss: 1.4096 - accuracy: 0.5746 - val_loss: 1.4171 - val_accuracy: 0.5810
Epoch 4/10
141/141 [==============================] - 9s 66ms/step - loss: 1.1743 - accuracy: 0.6472 - val_loss: 1.3235 - val_accuracy: 0.6075
Epoch 5/10
141/141 [==============================] - 9s 65ms/step - loss: 1.0020 - accuracy: 0.7019 - val_loss: 1.2813 - val_accuracy: 0.6255
Epoch 6/10
141/141 [==============================] - 9s 66ms/step - loss: 0.8713 - accuracy: 0.7389 - val_loss: 1.2789 - val_accuracy: 0.6357
Epoch 7/10
141/141 [==============================] - 9s 65ms/step - loss: 0.7700 - accuracy: 0.7733 - val_loss: 1.2854 - val_accuracy: 0.645

In [ ]:
#RF model with hyperparameter tuning and feature selection

ERROR: Could not find a version that satisfies the requirement scikitplot (from versions: none)
ERROR: No matching distribution found for scikitplot


ModuleNotFoundError: ignored